## Forecasting with sktime

### Reduction: from forecasting to regression

sktime provides a meta-estimator for this approach, which is:

* **modular** and **compatible with scikit-learn**, so that we can easily apply any scikit-learn regressor to solve our forecasting problem,
* **tuneable**, allowing us to tune hyper-parameters like the window length or strategy to generate forecasts
* **adaptive**, in the sense that it adapts the scikit-learn's estimator interface to that of a forecaster, making sure that we can tune and properly evaluate our model

In [ ]:
from sktime.forecasting.all import *
%matplotlib inline

In [ ]:
y = load_airline()
y_train, y_test = temporal_train_test_split(y, test_size=36)
print(y_train.shape[0], y_test.shape[0])

In [ ]:
from hcrystalball.wrappers import SarimaxWrapper
from hcrystalball.wrappers import ExponentialSmoothingWrapper
from hcrystalball.wrappers import get_sklearn_wrapper
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from hcrystalball.feature_extraction import SeasonalityTransformer

In [ ]:
# model = SarimaxWrapper(init_with_autoarima=True, autoarima_dict={"m":12})
# model = ExponentialSmoothingWrapper(trend="add", seasonal="multiplicative", seasonal_periods=12)
# model = get_sklearn_wrapper(RandomForestRegressor, n_estimators=100)
# model = pipeline = Pipeline([
#     ('seasonality', SeasonalityTransformer(freq='Y')),
#     ('model', get_sklearn_wrapper(RandomForestRegressor))
# ])
# model = get_sklearn_wrapper(ElasticNet)
model = pipeline = Pipeline([
    ('seasonality', SeasonalityTransformer(freq='Y')),
    ('model', get_sklearn_wrapper(ElasticNet))
])


In [ ]:
from sktime.forecasting.base._sktime import OptionalForecastingHorizonMixin
from sktime.forecasting.base._sktime import BaseSktimeForecaster
from sktime.forecasting.base._base import DEFAULT_ALPHA
import pandas as pd
import numpy as np

class HCrystalBallForecaster(OptionalForecastingHorizonMixin, BaseSktimeForecaster):
    def __init__(self, model):
        self.model = model
        self._is_fitted = False
        
    def fit(self, y_train, fh=None, X_train=None): 
        self._set_y_X(y_train, X_train)
        self._set_fh(fh)   
        
        y_train, X_train = self._adapt_fit_data(y_train, X_train)
                
        self.model.fit(X=X_train, y=y_train)        
        self._is_fitted = True
        
        return self
        
    def predict(self, fh=None, X=None, return_pred_int=False, alpha=DEFAULT_ALPHA):
        if return_pred_int:
            self._check_model_consistent_with_pred_int(alpha)
            
        self.check_is_fitted()
        self._set_fh(fh)
        
        X = self._adapt_predict_data(X)
                
        preds = self.model.predict(X=X)
        
        return self._convert_predictions(preds)
        

    @staticmethod
    def _adapt_fit_data(y_train, X_train): 
        X_train = X_train or pd.DataFrame()
        
        if type(y_train.index) in (pd.PeriodIndex, pd.DatetimeIndex):
            y = pd.Series(data=y_train.values,
                          index=y_train.index.to_timestamp() if type(y_train.index) is pd.PeriodIndex else y_train.index, 
                )
            X = pd.DataFrame(index=y.index)            
            if type(X_train.index) == type(X.index):
                X = X.merge(X_train, left_index=True, right_index=True)
            else:
                # different types, let's use X_train values with the datetime index inferred from y_train
                X = pd.DataFrame(data=X_train.values, index=X.index)
                    
        elif type(X_train.index) in (pd.PeriodIndex, pd.DatetimeIndex):
            #TODO check for X(2020-09-01, 2020-09-02, 2020-09-10), while y(2020-09-01 ... 2020-09-10)
            X = pd.DataFrame(
                    data=X_train.values, 
                    index=X_train.index.to_timestamp() if type(X_train.index) is pd.PeriodIndex else X_train.index
            )
            y_train = pd.Series(data=y_train.values, index=X.index)
        else:
            raise ValueError("At least one of y_train or X_train must have Period or DateTime index. "
                             f"You provided {type(X_train.index)} for X_train.index and {type(y_train.index)} for y_train.index")

        return y_train, X
        
    def _adapt_predict_data(self, X):
        _X = pd.DataFrame(index=self._fh.to_absolute(self._cutoff))
        _X.index = _X.index.to_timestamp()
        X = X or pd.DataFrame()
        
        if type(X.index) == type(_X.index):
            return _X.merge(X, left_index=True, right_index=True)
        else:
            return pd.DataFrame(data=X.values, index=_X.index)
            
    @staticmethod
    def _convert_predictions(preds):
        preds = preds.iloc[:,0]
        preds.index = preds.index.to_period()
        
        return preds
    
    #TODO: update per model/once the support is there for all models
    def _check_model_consistent_with_pred_int(alpha):
        raise NotImplemented("Full support for confidence intervals is not implemented.")    

In [ ]:
forecaster = HCrystalBallForecaster(model=model)
fh = np.arange(len(y_test)) + 1

In [ ]:
forecaster.fit(y_train, 
#                fh=fh
              )

In [ ]:
y_pred = forecaster.predict(fh)

In [ ]:
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

### Statistical forecasters

sktime has a number of statistical forecasting algorithms, based on implementations in statsmodels. For example, to use exponential smoothing with an additive trend component and multiplicative seasonality, we can write the following.

Note that since this is monthly data, the seasonal periodicity (sp), or the number of periods per year, is 12.

In [ ]:
# forecaster = HCrystalBallForecaster(model = ExponentialSmoothingWrapper(trend="add", seasonal="multiplicative", seasonal_periods=12))
forecaster = ExponentialSmoothing(trend="add", seasonal="multiplicative", sp=12)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

The exponential smoothing of state space model can also be automated similar
 to the [ets](https://www.rdocumentation.org/packages/forecast/versions/8.13/topics/ets) function in R.

In [ ]:
forecaster = HCrystalBallForecaster(model = ExponentialSmoothingWrapper(trend="add", seasonal="multiplicative", seasonal_periods=12))
# forecaster = ExponentialSmoothing(trend="add", seasonal="multiplicative", sp=12)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

In [ ]:
# from sktime.forecasting.ets import AutoETS
# forecaster = AutoETS(auto=True, sp=12, n_jobs=-1)
# forecaster.fit(y_train)
# y_pred = forecaster.predict(fh)
# plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
# smape_loss(y_test, y_pred)

Another common model is the ARIMA model. In sktime, we interface [pmdarima](https://github.com/alkaline-ml/pmdarima), a package for automatically selecting the best ARIMA model. This since searches over a number of possible model parametrisations, it may take a bit longer.

In [ ]:
# forecaster = HCrystalBallForecaster(model = SarimaxWrapper(init_with_autoarima=True, autoarima_dict={"m":12}))
forecaster = AutoARIMA(sp=12, suppress_warnings=True)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

In [ ]:
forecaster = HCrystalBallForecaster(model = SarimaxWrapper(init_with_autoarima=True, autoarima_dict={"m":12}))
# forecaster = AutoARIMA(sp=12, suppress_warnings=True)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

## Composite model building

sktime provides a modular API for composite model building for forecasting.

### Ensembling
Like scikit-learn, sktime provides a meta-forecaster to ensemble multiple forecasting algorithms. For example, we can combine different variants of exponential smoothing as follows:

In [ ]:
forecaster = EnsembleForecaster([
    ("ses", ExponentialSmoothing(seasonal="multiplicative", sp=12)),
    ("holt", ExponentialSmoothing(trend="add", damped=False, seasonal="multiplicative", sp=12)),
    ("damped", ExponentialSmoothing(trend="add", damped=True, seasonal="multiplicative", sp=12))
])
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

In [ ]:
forecaster = EnsembleForecaster([
    ("ses", HCrystalBallForecaster(ExponentialSmoothingWrapper(seasonal="multiplicative", seasonal_periods=12))),
    ("holt", HCrystalBallForecaster(ExponentialSmoothingWrapper(trend="add", damped=False, seasonal="multiplicative", seasonal_periods=12))),
    ("damped", HCrystalBallForecaster(ExponentialSmoothingWrapper(trend="add", damped=True, seasonal="multiplicative", seasonal_periods=12)))
])
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

## Tuning
In the `ReducedRegressionForecaster`, both the `window_length` and `strategy` arguments are hyper-parameters which we may want to optimise.

In [ ]:
forecaster = ReducedRegressionForecaster(regressor=regressor, window_length=15, strategy="recursive")
param_grid = {"window_length": [5, 10, 15]}

# we fit the forecaster on the initial window, and then use temporal cross-validation to find the optimal parameter
cv = SlidingWindowSplitter(initial_window=int(len(y_train) * 0.5))
gscv = ForecastingGridSearchCV(forecaster, cv=cv, param_grid=param_grid)
gscv.fit(y_train)
y_pred = gscv.predict(fh)

In [ ]:
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

In [ ]:
gscv.best_params_

Using scikit-learn's `GridSearchCV`, we can tune regressors imported from scikit-learn, in addition to tuning `window_length`.

In [ ]:
from sklearn.model_selection import GridSearchCV

# tuning the 'n_estimator' hyperparameter of RandomForestRegressor from scikit-learn
regressor_param_grid = {"n_estimators": [100, 200, 300]}
forecaster_param_grid = {"window_length": [5,10,15,20,25]}

# create a tunnable regressor with GridSearchCV
regressor = GridSearchCV(RandomForestRegressor(), param_grid=regressor_param_grid)
forecaster = ReducedRegressionForecaster(regressor, window_length=15, strategy="recursive")

cv = SlidingWindowSplitter(initial_window=int(len(y_train) * 0.5))
gscv = ForecastingGridSearchCV(forecaster, cv=cv, param_grid=forecaster_param_grid)

gscv.fit(y_train)
y_pred = gscv.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

In [ ]:
print(gscv.best_params_, gscv.best_forecaster_.regressor_.best_params_)

To access performance on a particular metric during tuning, we can use the `scoring` argument of `ForecastingGridSearchCV`.

In [ ]:
gscv = ForecastingGridSearchCV(forecaster, cv=cv, param_grid=forecaster_param_grid, scoring=sMAPE())
gscv.fit(y_train)
pd.DataFrame(gscv.cv_results_)

### Detrending
Note that so far the reduction approach above does not take any seasonal or trend into account, but we can easily specify a pipeline which first detrends the data.

sktime provides a generic detrender, a transformer which uses any forecaster and returns the in-sample residuals of the forecaster's predicted values. For example, to remove the linear trend of a time series, we can write:

In [ ]:
# liner detrending
forecaster = PolynomialTrendForecaster(degree=1)
transformer = Detrender(forecaster=forecaster)
yt = transformer.fit_transform(y_train)

# internally, the Detrender uses the in-sample predictions of the PolynomialTrendForecaster
forecaster = PolynomialTrendForecaster(degree=1)
fh_ins = -np.arange(len(y_train)) # in-sample forecasting horizon
y_pred = forecaster.fit(y_train).predict(fh=fh_ins)

plot_series(y_train, y_pred, yt, labels=["y_train", "fitted linear trend", "residuals"]);

### Pipelining

Let's use the detrender in a pipeline together with de-seasonalisation. Note that in forecasting, when we apply data transformations before fitting, we need to apply the inverse transformation to the predicted values. For this purpose, we provide the following pipeline class:

In [ ]:
forecaster = TransformedTargetForecaster([
    ("deseasonalise", Deseasonalizer(model="multiplicative", sp=12)),
    ("detrend", Detrender(forecaster=PolynomialTrendForecaster(degree=1))),
    ("forecast", ReducedRegressionForecaster(regressor=regressor, window_length=12, strategy="recursive"))
])
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);
smape_loss(y_test, y_pred)

Of course, we could try again to optimise the hyper-parameters of components of the pipeline.

Below we discuss two other aspects of forecasting: online learning, where we want to dynamically update forecasts as new data comes in, and prediction intervals, which allow us to quantify the uncertainty of our forecasts.

## Dynamic forecasts

For model evaluation, we sometimes want to evaluate multiple forecasts, using temporal cross-validation with a sliding window over the test data. For this purpose, all forecasters in sktime have a `update_predict` method. Here we make repeated single-step ahead forecasts over the test set.

Note that the forecasting task is changed: while we still make 36 predictions, we do not predict 36 steps ahead, but instead make 36 single-step-ahead predictions.

In [ ]:
forecaster = NaiveForecaster(strategy="last")
forecaster.fit(y_train)
cv = SlidingWindowSplitter(fh=1)
y_pred = forecaster.update_predict(y_test, cv)
smape_loss(y_test, y_pred)
plot_series(y_train, y_test, y_pred);

For a single update, you can use the `update` method.

## Prediction intervals
So far, we've only looked at point forecasts. In many cases, we're also interested in prediction intervals. sktime's interface support prediction intervals, but we haven't implemented them for all algorithms yet.

Here, we use the Theta forecasting algorithm:

In [ ]:
forecaster = ThetaForecaster(sp=12)
forecaster.fit(y_train)
alpha = 0.05  # 95% prediction intervals
y_pred, pred_ints = forecaster.predict(fh, return_pred_int=True, alpha=alpha)
smape_loss(y_test, y_pred)

In [ ]:
fig, ax = plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
ax.fill_between(ax.get_lines()[-1].get_xdata(), pred_ints["lower"], pred_ints["upper"],
                alpha=0.2, color=ax.get_lines()[-1].get_c(), label=f"{1 - alpha}% prediction intervals")
ax.legend();

## Summary

As we have seen, in order to make forecasts, we need to first specify (or build) a model, then fit it to the training data, and finally call predict to generate forecasts for the given forecasting horizon.

* sktime comes with several forecasting algorithms (or forecasters) and tools for composite model building. All forecaster share a common interface. Forecasters are trained on a single series of data and make forecasts for the provided forecasting horizon.

* sktime has a number of statistical forecasting algorithms, based on implementations in statsmodels. For example, to use exponential smoothing with an additive trend component and multiplicative seasonality, we can write the following.


## Useful resources
* For more details, take a look at [our paper on forecasting with sktime](https://arxiv.org/abs/2005.08067) in which we discuss the forecasting API in more detail and use it to replicate and extend the M4 study.
* For a good introduction to forecasting, see [Hyndman, Rob J., and George Athanasopoulos. Forecasting: principles and practice. OTexts, 2018](https://otexts.com/fpp2/).
* For comparative benchmarking studies/forecasting competitions, see the [M4 competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128) and the currently running [M5 competition](https://www.kaggle.com/c/m5-forecasting-accuracy/overview).